In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
import os
from darkflow.net.build import TFNet
import random as rd
from sklearn.metrics import f1_score

# Parameter Setting

## for both

In [2]:
expansion_rate = 2
path_test_data = "../training/"
#path_test_data = "labels/cup/images/"
path_result_folder = "mlp-training/result/"
path_premlp = "mlp-training/result/premlp/"
path_original = "mlp-training/result/original/"
path_cup = "labels/cup/images/"
path_cellphone = "labels/cellphone/images/"
path_paper = "labels/paper/images/"
path_mlp_training = "mlp-training/"



## for YOLO

In [3]:
options = {"model": "cfg/truck-yolo.cfg", "load": -1, "gpu":0.75,"threshold":0.9}
#options = {"model": "cfg/truck-yolo.cfg", "load": 56500, "gpu":0.75}
path_yolo_test = "labels/cellphone/images/"

## for MLP

In [4]:
path_mlp_sample_image = "mlp-training/cellphone-0.jpg"
path_mlp_csv = "mlp-training/csv/truck_driver.csv"
path_mlp_model = "mlp-training/saved_models/model-800.ckpt"
size_mlp_data = (100,100)

# Function

In [5]:
def relu(num):
    if num > 0:
        return num
    else:
        return 0
    
def expansion(o_img,bbox,rate):
    img = o_img.copy()
    width = bbox[1] - bbox[0]
    height = bbox[3] - bbox[2]
    center = [bbox[0]+width/2,bbox[2]+height/2]
    
    new_bbox = [relu(round(center[0] - (width/2)*rate)), \
                relu(round(center[0] + (width/2)*rate)), \
                relu(round(center[1] - (height/2)*rate)), \
                relu(round(center[1] + (height/2)*rate)) ]
    img = img[new_bbox[2]:new_bbox[3], new_bbox[0]:new_bbox[1]]
    #print(center)
    #print(new_bbox)
    return img

def get_yolo_info(imgcv):
    result = tfnet.return_predict(imgcv)
    if len(result) != 0:
        result = result[0]
        info = {'xmin':result["topleft"]["x"],
                'xmax':result["bottomright"]["x"],
                'ymin':result["topleft"]["y"],
                'ymax':result["bottomright"]["y"],
                'label':result["label"]
               }
        return info
    else:
        return False
    
def pre_mlp(o_img,info,expansion_rate):
    #img = img[info['ymin']:info['ymax'],info['xmin']:info['xmax']]
    img = expansion(o_img, [info['xmin'],info['xmax'], info['ymin'], info['ymax']], expansion_rate)
    #img = expansion(img, [info['xmin'],info['xmax'], img.shape[0]-info['ymax'], img.shape[0]-info['ymin']], expansion_rate)
    return img

def draw(img,info):
    output = img.copy()
    cv2.rectangle(output, (info['xmin'], info['ymin']), (info['xmax'], info['ymax']), (0,0,255),2)
    text_x, text_y = info['xmax']+10 , info['ymin']+10 
    cv2.putText(output, info['label'], (text_x, text_y),cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2, cv2.LINE_AA)
    return output

def detect(imgcv):
    result = tfnet.return_predict(imgcv)
    #print(type(result[0]["topleft"]["x"]))
    if len(result) != 0:
        result = result[0]
        cv2.rectangle(imgcv, 
            (result["topleft"]["x"], result["topleft"]["y"]), 
            (result["bottomright"]["x"], 
            result["bottomright"]["y"]), 
            (0, 255, 0), 4)
        text_x, text_y = result["topleft"]["x"] - 10, result["topleft"]["y"] - 10
        cv2.putText(imgcv, result["label"], (text_x, text_y),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)
        #print(result)
        # cv2.imshow('Test',imgcv)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()
        return imgcv
    else:
        return np.array([])


    


# MLP + YOLO Setting

## MLP Setting

In [6]:
img = cv2.imread(path_mlp_sample_image,0)
img2 = cv2.resize(img, size_mlp_data)
img2 = img2.flatten()
print(img2.shape)

(10000,)


In [7]:
imv_csv = pd.read_csv(path_mlp_csv)
class_y = pd.get_dummies(imv_csv['class'], '').as_matrix()#將label做 one_hot encoding
x_train_list, y_train = imv_csv, class_y #資料切割

C:\Users\Yuga\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


### mlp model setting

In [8]:
tf.reset_default_graph()

In [9]:
#### define placeholder ####
input_data = tf.placeholder(dtype=tf.float32, 
                           shape=[None, img2.shape[0]],
                           name='input_data') #用來接 feature 資料進入 tensorflow 

y_true = tf.placeholder(dtype=tf.float32, 
                        shape=[None, y_train.shape[1]],
                        name='y_true') #用來接 label 資料進入 tensorflow 

In [10]:
#### define variables(weight/bias) ####
x1 = tf.layers.dense(input_data, 256, activation=tf.nn.sigmoid, name='hidden1') #第一層hidden layer
x2 = tf.layers.dense(x1, 128, activation=tf.nn.sigmoid, name='hidden2') #第二層hidden layer
x3 = tf.layers.dense(x2, 64, activation=tf.nn.sigmoid, name='hidden3')#第三層hidden layer
out = tf.layers.dense(x3, y_train.shape[1], name='output')# output layer

y_pred = out

In [11]:
#### calculate loss ####
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [12]:
#### optimize variables ####
opt = tf.train.GradientDescentOptimizer(learning_rate=0.001)
update = opt.minimize(loss)

In [13]:
#### init ####
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [14]:
tf.global_variables() ## 檢查 graph 裏的 global variables

[<tf.Variable 'hidden1/kernel:0' shape=(10000, 256) dtype=float32_ref>,
 <tf.Variable 'hidden1/bias:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'hidden2/kernel:0' shape=(256, 128) dtype=float32_ref>,
 <tf.Variable 'hidden2/bias:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable 'hidden3/kernel:0' shape=(128, 64) dtype=float32_ref>,
 <tf.Variable 'hidden3/bias:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'output/kernel:0' shape=(64, 2) dtype=float32_ref>,
 <tf.Variable 'output/bias:0' shape=(2,) dtype=float32_ref>]

### mlp load model

In [15]:
'''rerun the graph first:
先回到上面將一開始和graph有關的cell先重新執行一遍，將graph架構先建立起來(這樣讀進來的參數才有對應位置存放)'''

sess = tf.Session()
saver = tf.train.Saver()
saver.restore(sess, path_mlp_model) #到我們剛剛存檔的路徑將檔案叫出來，放入graph中對應的參數位置

INFO:tensorflow:Restoring parameters from mlp-training/saved_models/model-800.ckpt


In [16]:
tf.global_variables()

[<tf.Variable 'hidden1/kernel:0' shape=(10000, 256) dtype=float32_ref>,
 <tf.Variable 'hidden1/bias:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'hidden2/kernel:0' shape=(256, 128) dtype=float32_ref>,
 <tf.Variable 'hidden2/bias:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable 'hidden3/kernel:0' shape=(128, 64) dtype=float32_ref>,
 <tf.Variable 'hidden3/bias:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'output/kernel:0' shape=(64, 2) dtype=float32_ref>,
 <tf.Variable 'output/bias:0' shape=(2,) dtype=float32_ref>]

## YOLO Setting

In [17]:
tfnet = TFNet(options)

Parsing cfg/truck-yolo.cfg
Loading None ...
Finished in 0.0s

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
       |        | input                            | (?, 608, 608, 3)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 608, 608, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 304, 304, 32)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 304, 304, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 152, 152, 64)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Init  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 152, 152, 64)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 76, 76, 128)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 76, 76, 256)
 Init  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 76, 76, 128)
 Init  | 

# Main (YOLO)

## Test original data

In [26]:
yolo_predict = []
for file in os.listdir(path_yolo_test):
    img = cv2.imread(path_yolo_test+file)
    imgcv = detect(img)
    if len(imgcv) != 0:
        yolo_predict.append(1)
    else:
        yolo_predict.append(0)
#print(yolo_predict)
print('f1-score:',f1_score(np.full((1,len(yolo_predict)),1)[0], yolo_predict,average='binary'))

f1-score: 0.7820069204152249


In [25]:
yolo_predict = []
for file in os.listdir(path_new):
    img = cv2.imread(path_new+file)
    imgcv = detect(img)
    if len(imgcv) != 0:
        yolo_predict.append(1)
    else:
        yolo_predict.append(0)
#print(yolo_predict)
#print('f1-score:',f1_score(np.full((1,len(yolo_predict)),1)[0], yolo_predict,average='binary'))
#print(sum(yolo_predict), len(yolo_predict) )
print("誤差率:", 100*sum(yolo_predict)/len(yolo_predict) ,"%")

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
f1-score: 0.10441767068273092
13 236
誤差率: 5.508474576271187 %


## Read Video

In [15]:
total = 0
count = 0
timeF = 50
path_all_video = "../完整影片/"
path_save_folder = path_video_folder+'result/'
file_list = os.listdir(path_video_folder)
if not os.path.exists(path_save_folder):
    os.makedirs(path_save_folder)

In [13]:
for file in file_list:
    cap = cv2.VideoCapture(path_video_folder+folder_name+file)
    c = 0
    # 以迴圈從影片檔案讀取影格，並顯示出來
    while(cap.isOpened()):
        ret, frame = cap.read()
        if frame is None:
            break

        if(c%timeF==0):
            #print(type(frame))
            origin = frame[35:,100:300]
#             cv2.imshow('123',origin)
#             if cv2.waitKey(1) & 0xFF == ord('q'):
#                 break
            imgcv = detect(origin)
            if len(imgcv) != 0:
                count += 1
                cv2.imwrite(path_save_folder+file[:file.index('.')]+'.jpg',imgcv)
            total += 1
        c += 1
    cap.release()
    cv2.destroyAllWindows()
    print('Object detected files:',count,'/',total)

NameError: name 'file_list' is not defined

# Main (MLP+YOLO)

## Test original data

In [ ]:
yolo_predict = []
for file in os.listdir(path_yolo_test):
    img = cv2.imread(path_yolo_test+file)
    imgcv = detect(img)
    if len(imgcv) != 0:
        yolo_predict.append(1)
    else:
        yolo_predict.append(0)
#print(yolo_predict)
print('f1-score:',f1_score(np.full((1,len(yolo_predict)),1)[0], yolo_predict,average='binary'))

## Read Image

In [22]:
dic = {'normal_detect':0,'normal_undetect':0,'abnormal_detect':0,'abnormal_undetect':0}
correct = 0
total_result = [0,0,0,0]
data_number = 1000
tmp = os.listdir(path_cellphone)
dir_list = []
for i in range(len(tmp)):
    if 'jpg' in tmp[i]:
        dir_list.append(tmp[i])

In [23]:
# for file in dir_list:
#     img = cv2.imread(path_test_data+file)
#     img = detect(img)
#     if len(img) !=0:
#         #print(yolo_info)
#         cv2.imwrite(path_original+file, img)

In [24]:
dir_list

['0.jpg',
 '1.jpg',
 '10.jpg',
 '100.jpg',
 '101.jpg',
 '102.jpg',
 '103.jpg',
 '104.jpg',
 '105.jpg',
 '106.jpg',
 '107.jpg',
 '108.jpg',
 '109.jpg',
 '11.jpg',
 '110.jpg',
 '111.jpg',
 '112.jpg',
 '113.jpg',
 '114.jpg',
 '115.jpg',
 '116.jpg',
 '117.jpg',
 '118.jpg',
 '119.jpg',
 '12.jpg',
 '120.jpg',
 '121.jpg',
 '122.jpg',
 '123.jpg',
 '124.jpg',
 '125.jpg',
 '126.jpg',
 '127.jpg',
 '128.jpg',
 '129.jpg',
 '13.jpg',
 '130.jpg',
 '131.jpg',
 '132.jpg',
 '133.jpg',
 '134.jpg',
 '135.jpg',
 '136.jpg',
 '137.jpg',
 '138.jpg',
 '139.jpg',
 '14.jpg',
 '140.jpg',
 '141.jpg',
 '142.jpg',
 '143.jpg',
 '144.jpg',
 '145.jpg',
 '146.jpg',
 '147.jpg',
 '148.jpg',
 '149.jpg',
 '15.jpg',
 '150.jpg',
 '151.jpg',
 '152.jpg',
 '153.jpg',
 '154.jpg',
 '155.jpg',
 '156.jpg',
 '157.jpg',
 '158.jpg',
 '159.jpg',
 '16.jpg',
 '160.jpg',
 '161.jpg',
 '162.jpg',
 '163.jpg',
 '164.jpg',
 '165.jpg',
 '166.jpg',
 '167.jpg',
 '168.jpg',
 '169.jpg',
 '17.jpg',
 '170.jpg',
 '171.jpg',
 '172.jpg',
 '173.jpg',
 '17

In [25]:
count = 0
for file in dir_list:
    img = cv2.imread(path_cellphone+file)
    yolo_info = get_yolo_info(img)
    if yolo_info != False:
        print(file, yolo_info)
        #cv2.imwrite(path_original+file, draw(img, yolo_info))
        #expansion process
        bbox = pre_mlp(img,yolo_info,expansion_rate)
        cv2.imwrite(path_premlp+file, bbox)
        #preprocess
        bbox = cv2.resize(bbox,size_mlp_data)
        bbox = cv2.cvtColor(bbox, cv2.COLOR_BGR2GRAY)
        bbox = bbox.flatten()
        #print(bbox.shape)
        #tmpx
        tmpx = np.array([]).reshape((0, img2.shape[0]))
        tmpx = np.row_stack([tmpx, bbox.flatten()])
        tmpx /= 255
        #tmpy
        tmpy = np.array([]).reshape((0, y_train.shape[1]))
        tmpy = np.row_stack([tmpy, class_y[0]])
        tr_pred = sess.run([y_pred], feed_dict={input_data:tmpx})
        if tr_pred[0][0][0] > tr_pred[0][0][1]:
            print(tr_pred)
            img = draw(img, yolo_info)
            cv2.imwrite(path_result_folder+file, img)
            count += 1
print("誤差率:", 100*count/len(dir_list),"%")

0.jpg {'xmin': 328, 'xmax': 375, 'ymin': 261, 'ymax': 308, 'label': 'cellphone'}
[array([[ 1.9116882, -2.1895523]], dtype=float32)]
1.jpg {'xmin': 328, 'xmax': 375, 'ymin': 261, 'ymax': 308, 'label': 'cellphone'}
[array([[ 1.9143988, -2.1919494]], dtype=float32)]
10.jpg {'xmin': 90, 'xmax': 136, 'ymin': 370, 'ymax': 430, 'label': 'cellphone'}
[array([[ 0.08121147, -0.56160843]], dtype=float32)]
100.jpg {'xmin': 278, 'xmax': 326, 'ymin': 208, 'ymax': 256, 'label': 'cellphone'}
[array([[ 1.0706854, -1.4375196]], dtype=float32)]
101.jpg {'xmin': 278, 'xmax': 326, 'ymin': 217, 'ymax': 264, 'label': 'cellphone'}
[array([[-0.03508101, -0.46273607]], dtype=float32)]
102.jpg {'xmin': 129, 'xmax': 173, 'ymin': 340, 'ymax': 416, 'label': 'cellphone'}
[array([[ 2.2652695, -2.4942844]], dtype=float32)]
103.jpg {'xmin': 127, 'xmax': 171, 'ymin': 346, 'ymax': 424, 'label': 'cellphone'}
[array([[ 2.39477 , -2.613116]], dtype=float32)]
104.jpg {'xmin': 127, 'xmax': 171, 'ymin': 346, 'ymax': 423, 'labe

164.jpg {'xmin': 345, 'xmax': 392, 'ymin': 123, 'ymax': 167, 'label': 'cellphone'}
[array([[ 3.7086294, -3.7741778]], dtype=float32)]
165.jpg {'xmin': 342, 'xmax': 386, 'ymin': 125, 'ymax': 164, 'label': 'cellphone'}
[array([[ 3.4719007, -3.5729043]], dtype=float32)]
166.jpg {'xmin': 345, 'xmax': 384, 'ymin': 136, 'ymax': 173, 'label': 'cellphone'}
[array([[ 3.5403588, -3.6273348]], dtype=float32)]
169.jpg {'xmin': 196, 'xmax': 241, 'ymin': 339, 'ymax': 386, 'label': 'cellphone'}
[array([[ 1.05442  , -1.4251102]], dtype=float32)]
170.jpg {'xmin': 129, 'xmax': 173, 'ymin': 361, 'ymax': 447, 'label': 'cellphone'}
[array([[ 0.7113157, -1.1210455]], dtype=float32)]
171.jpg {'xmin': 303, 'xmax': 350, 'ymin': 174, 'ymax': 220, 'label': 'cellphone'}
[array([[ 2.0632012, -2.3224854]], dtype=float32)]
172.jpg {'xmin': 126, 'xmax': 168, 'ymin': 344, 'ymax': 429, 'label': 'cellphone'}
[array([[ 1.5282097, -1.8436666]], dtype=float32)]
173.jpg {'xmin': 123, 'xmax': 166, 'ymin': 356, 'ymax': 446, '

80.jpg {'xmin': 346, 'xmax': 389, 'ymin': 147, 'ymax': 185, 'label': 'cellphone'}
[array([[ 2.31152  , -2.5423663]], dtype=float32)]
81.jpg {'xmin': 330, 'xmax': 374, 'ymin': 159, 'ymax': 197, 'label': 'cellphone'}
[array([[ 1.8252276, -2.1129835]], dtype=float32)]
82.jpg {'xmin': 349, 'xmax': 387, 'ymin': 158, 'ymax': 192, 'label': 'cellphone'}
[array([[ 1.0940379, -1.4645603]], dtype=float32)]
83.jpg {'xmin': 349, 'xmax': 389, 'ymin': 155, 'ymax': 190, 'label': 'cellphone'}
[array([[ 0.73658335, -1.14241   ]], dtype=float32)]
84.jpg {'xmin': 344, 'xmax': 384, 'ymin': 158, 'ymax': 192, 'label': 'cellphone'}
[array([[ 0.11543451, -0.5955532 ]], dtype=float32)]
85.jpg {'xmin': 334, 'xmax': 377, 'ymin': 159, 'ymax': 195, 'label': 'cellphone'}
[array([[ 0.71147233, -1.1194191 ]], dtype=float32)]
86.jpg {'xmin': 332, 'xmax': 375, 'ymin': 161, 'ymax': 197, 'label': 'cellphone'}
[array([[ 1.655963 , -1.9559885]], dtype=float32)]
87.jpg {'xmin': 334, 'xmax': 377, 'ymin': 161, 'ymax': 199, 'la

## Read Video

In [18]:
video_path = "../FullVideo/DISK1/"
video_list = os.listdir(video_path)

In [19]:
print(video_list)
print(len(video_list))

['20190318']
1


In [20]:
for d in video_list:
    date = d
    path_video = video_path+d+"/"
    path_video_premlp = path_video+"premlp/"
    path_video_result_folder = "../FullVideo/results/"+date+"/"
    path_video_result_folder2 = "../FullVideo/results/"+date+"/"+"original/"
    path_video_yolo = path_video+"yolo/"

#     if not os.path.isdir(path_video_premlp):
#         os.makedirs(path_video_premlp)
    if not os.path.isdir(path_video_result_folder):
        os.makedirs(path_video_result_folder)
    if not os.path.isdir(path_video_result_folder2):
        os.makedirs(path_video_result_folder2)

    video_list = []
    for video in os.listdir(path_video):
        if 'avi' in video:
            video_list.append(video)
    print(video_list)

    total = 0
    count = 0
    switch = 1
    move_list = []
    stop_count = []
    move_count = []
    status_list = []
    stop_rate = 0.8
    start_rate = 0.2
    standard = 0
    vnum = 1
    for video in video_list:
        print('Now processing:', video, vnum,"/",len(video_list))
        vnum+=1
        cap = cv2.VideoCapture(path_video+video)
        timeF = 100
        c = 0
        # 以迴圈從影片檔案讀取影格，並顯示出來
        while(cap.isOpened()):
            ret, frame = cap.read()
            if frame is None:
                break





            if(c%timeF==0):
                #print(c)
                #判斷車子是否行進間   
    #             new_img = frame[100:250,350:500]
                new_img = frame[100:250,360:500]
                if c==0:
                    ret,previous = cv2.threshold(new_img,127,255,cv2.THRESH_BINARY)
                else:
                    ret,current = cv2.threshold(new_img,127,255,cv2.THRESH_BINARY)
                    minus = cv2.subtract(current,previous)
                    previous = current.copy()
                    move = np.sum(minus/255)

    #                 cv2.imshow('mytitle',minus)
    #                 #print(move)
    #                 if cv2.waitKey(1) & 0xFF == ord('q'):
    #                     break
    #                 if c== 3000:
    #                     cap.release()
    #                     cv2.destroyAllWindows()

                    if move < 2000:
                        status_list.append(0)
                    else:
                        status_list.append(1)

                    if len(status_list) == 10:
                        if sum(status_list) < 10*(1-stop_rate):
                            switch = 0
                        if sum(status_list) > 10*start_rate:
                            switch = 1
                        #print(status_list, switch)
                        status_list.pop(0)



                #print(type(frame))
                img = frame[:,5:300]


                if switch == 1:
                    yolo_info = get_yolo_info(img)
                    if yolo_info != False:
                        #print(yolo_info)
                        #yolo = draw(img, yolo_info)
                        #cv2.imwrite(path_video_yolo+str(c)+".jpg", yolo)
                        #cv2.imwrite(path_original+file, draw(img, yolo_info))
                        #expansion process
                        bbox = pre_mlp(img,yolo_info,expansion_rate)
                        #cv2.imwrite(path_video_premlp+video[:video.index('.')]+'_'+str(count)+".jpg", bbox)
                        #print(path_video_premlp+str(count)+".jpg")
                        #preprocess
                        bbox = cv2.resize(bbox,size_mlp_data)
                        bbox = cv2.cvtColor(bbox, cv2.COLOR_BGR2GRAY)
                        #print(bbox.shape)
                        bbox = bbox.flatten()
                        #print(bbox.shape)
                        #tmpx
                        tmpx = np.array([]).reshape((0, img2.shape[0]))
                        tmpx = np.row_stack([tmpx, bbox.flatten()])
                        tmpx /= 255
                        #tmpy
                        tmpy = np.array([]).reshape((0, y_train.shape[1]))
                        tmpy = np.row_stack([tmpy, class_y[0]])
                        tr_pred = sess.run([y_pred], feed_dict={input_data:tmpx})
                        if tr_pred[0][0][0] > tr_pred[0][0][1]:
                        #if tr_pred[0][0][0] > 0.5:
                            print(tr_pred)
                            img = draw(img, yolo_info)
                            cv2.imwrite(path_video_result_folder+video[:video.index('.')]+'_'+str(count)+".jpg", img)
                            cv2.imwrite(path_video_result_folder2+video[:video.index('.')]+'_'+str(count)+".jpg", frame)
                            print(path_video_result_folder+video[:video.index('.')]+'_'+str(count)+".jpg")
                        count += 1
            c += 1
        print('total count:', count)
        print('----------------------------')
        cap.release()
        cv2.destroyAllWindows()
        #print('Object detected files:',count,'/',total)

['0000000000000000-180712-180639-182140-10u402000000.avi', '0000000000000000-180712-182140-183640-10u402000000.avi', '0000000000000000-180712-183640-185140-10u402000000.avi', '0000000000000000-180712-185140-190640-10u402000000.avi', '0000000000000000-180712-190640-192140-10u402000000.avi', '0000000000000000-180712-192140-193640-10u402000000.avi', '0000000000000000-180712-193640-195140-10u402000000.avi', '0000000000000000-180712-195140-200640-10u402000000.avi', '0000000000000000-180712-200640-202140-10u402000000.avi', '0000000000000000-180712-202140-203640-10u402000000.avi', '0000000000000000-180712-203640-205140-10u402000000.avi', '0000000000000000-180712-205140-210640-10u402000000.avi', '0000000000000000-180712-210640-212140-10u402000000.avi', '0000000000000000-180712-212140-213640-10u402000000.avi', '0000000000000000-180712-213640-215140-10u402000000.avi', '0000000000000000-180712-215140-220640-10u402000000.avi', '0000000000000000-180712-220640-222140-10u402000000.avi', '000000000000

total count: 0
----------------------------
Now processing: 0000000000000000-180713-001500-003000-10u402000000.avi 2 / 68
total count: 0
----------------------------
Now processing: 0000000000000000-180713-003000-004500-10u402000000.avi 3 / 68
total count: 0
----------------------------
Now processing: 0000000000000000-180713-004500-010000-10u402000000.avi 4 / 68
total count: 0
----------------------------
Now processing: 0000000000000000-180713-010000-011500-10u402000000.avi 5 / 68
total count: 0
----------------------------
Now processing: 0000000000000000-180713-011500-013000-10u402000000.avi 6 / 68
total count: 0
----------------------------
Now processing: 0000000000000000-180713-013000-014500-10u402000000.avi 7 / 68
total count: 0
----------------------------
Now processing: 0000000000000000-180713-014500-020000-10u402000000.avi 8 / 68
total count: 0
----------------------------
Now processing: 0000000000000000-180713-020000-021500-10u402000000.avi 9 / 68
total count: 0
---------

[array([[ 0.5191268, -0.9576539]], dtype=float32)]
../FullVideo/results/20180713/0000000000000000-180713-093000-094500-10u402000000_32.jpg
total count: 36
----------------------------
Now processing: 0000000000000000-180713-094500-100000-10u402000000.avi 40 / 68
[array([[-0.21093604, -0.30896747]], dtype=float32)]
../FullVideo/results/20180713/0000000000000000-180713-094500-100000-10u402000000_36.jpg
[array([[-0.01541816, -0.48284006]], dtype=float32)]
../FullVideo/results/20180713/0000000000000000-180713-094500-100000-10u402000000_37.jpg
[array([[ 0.9521913, -1.344537 ]], dtype=float32)]
../FullVideo/results/20180713/0000000000000000-180713-094500-100000-10u402000000_38.jpg
[array([[-0.25265995, -0.27376986]], dtype=float32)]
../FullVideo/results/20180713/0000000000000000-180713-094500-100000-10u402000000_39.jpg
[array([[ 0.8999363, -1.2982398]], dtype=float32)]
../FullVideo/results/20180713/0000000000000000-180713-094500-100000-10u402000000_40.jpg
total count: 41
--------------------

[array([[ 3.9622743, -4.031111 ]], dtype=float32)]
../FullVideo/results/20180714/0000000000000000-180714-000000-001459-10u402000000_1.jpg
[array([[ 3.7665164, -3.8597496]], dtype=float32)]
../FullVideo/results/20180714/0000000000000000-180714-000000-001459-10u402000000_2.jpg
[array([[ 3.956454 , -4.0274496]], dtype=float32)]
../FullVideo/results/20180714/0000000000000000-180714-000000-001459-10u402000000_3.jpg
[array([[ 3.9487417, -4.0201516]], dtype=float32)]
../FullVideo/results/20180714/0000000000000000-180714-000000-001459-10u402000000_4.jpg
[array([[ 4.0799437, -4.137011 ]], dtype=float32)]
../FullVideo/results/20180714/0000000000000000-180714-000000-001459-10u402000000_5.jpg
[array([[ 3.8689306, -3.9486763]], dtype=float32)]
../FullVideo/results/20180714/0000000000000000-180714-000000-001459-10u402000000_6.jpg
[array([[ 3.882363 , -3.9604867]], dtype=float32)]
../FullVideo/results/20180714/0000000000000000-180714-000000-001459-10u402000000_7.jpg
[array([[ 3.8155296, -3.9013484]],

total count: 25
----------------------------
Now processing: 0000000000000000-180714-113000-114500-10u402000000.avi 47 / 74
total count: 25
----------------------------
Now processing: 0000000000000000-180714-114500-120000-10u402000000.avi 48 / 74
total count: 25
----------------------------
Now processing: 0000000000000000-180714-120000-121500-10u402000000.avi 49 / 74
total count: 25
----------------------------
Now processing: 0000000000000000-180714-121500-123000-10u402000000.avi 50 / 74
total count: 25
----------------------------
Now processing: 0000000000000000-180714-123000-124500-10u402000000.avi 51 / 74
total count: 25
----------------------------
Now processing: 0000000000000000-180714-124500-130000-10u402000000.avi 52 / 74
[array([[ 4.428606, -4.443286]], dtype=float32)]
../FullVideo/results/20180714/0000000000000000-180714-124500-130000-10u402000000_25.jpg
[array([[ 3.9600244, -4.0288296]], dtype=float32)]
../FullVideo/results/20180714/0000000000000000-180714-124500-130000-

...

total count: 9
----------------------------
Now processing: 0000000000000000-180716-010000-011500-10u402000000.avi 5 / 45
total count: 9
----------------------------
Now processing: 0000000000000000-180716-011500-013000-10u402000000.avi 6 / 45
total count: 9
----------------------------
Now processing: 0000000000000000-180716-013000-014500-10u402000000.avi 7 / 45
total count: 9
----------------------------
Now processing: 0000000000000000-180716-014500-020000-10u402000000.avi 8 / 45
total count: 9
----------------------------
Now processing: 0000000000000000-180716-020000-021500-10u402000000.avi 9 / 45
total count: 9
----------------------------
Now processing: 0000000000000000-180716-021500-023000-10u402000000.avi 10 / 45
total count: 9
----------------------------
Now processing: 0000000000000000-180716-023000-024500-10u402000000.avi 11 / 45
total count: 9
----------------------------
Now processing: 0000000000000000-180716-024500-030000-10u402000000.avi 12 / 45
total count: 9
------

total count: 0
----------------------------
Now processing: 0000000000000000-180717-001500-003000-10u402000000.avi 2 / 74
total count: 0
----------------------------
Now processing: 0000000000000000-180717-003000-004500-10u402000000.avi 3 / 74
total count: 0
----------------------------
Now processing: 0000000000000000-180717-004500-010000-10u402000000.avi 4 / 74
total count: 0
----------------------------
Now processing: 0000000000000000-180717-010000-011500-10u402000000.avi 5 / 74
total count: 0
----------------------------
Now processing: 0000000000000000-180717-011500-013000-10u402000000.avi 6 / 74
total count: 0
----------------------------
Now processing: 0000000000000000-180717-013000-014500-10u402000000.avi 7 / 74
total count: 0
----------------------------
Now processing: 0000000000000000-180717-014500-020000-10u402000000.avi 8 / 74
total count: 0
----------------------------
Now processing: 0000000000000000-180717-020000-021500-10u402000000.avi 9 / 74
total count: 0
---------

[array([[ 2.3691552, -2.611026 ]], dtype=float32)]
../FullVideo/results/20180717/0000000000000000-180717-094500-100000-10u402000000_28.jpg
[array([[ 1.0820544, -1.4669567]], dtype=float32)]
../FullVideo/results/20180717/0000000000000000-180717-094500-100000-10u402000000_29.jpg
[array([[ 1.6449945, -1.9714794]], dtype=float32)]
../FullVideo/results/20180717/0000000000000000-180717-094500-100000-10u402000000_30.jpg
[array([[ 1.0281435, -1.4228288]], dtype=float32)]
../FullVideo/results/20180717/0000000000000000-180717-094500-100000-10u402000000_31.jpg
[array([[ 2.79723 , -2.999766]], dtype=float32)]
../FullVideo/results/20180717/0000000000000000-180717-094500-100000-10u402000000_32.jpg
[array([[ 3.3225439, -3.4625285]], dtype=float32)]
../FullVideo/results/20180717/0000000000000000-180717-094500-100000-10u402000000_33.jpg
[array([[ 2.6754198, -2.8868542]], dtype=float32)]
../FullVideo/results/20180717/0000000000000000-180717-094500-100000-10u402000000_34.jpg
[array([[ 1.8706151, -2.17494

total count: 92
----------------------------
Now processing: 0000000000000000-180717-144500-150000-10u402000000.avi 60 / 74
total count: 92
----------------------------
Now processing: 0000000000000000-180717-150000-151500-10u402000000.avi 61 / 74
total count: 92
----------------------------
Now processing: 0000000000000000-180717-151500-153000-10u402000000.avi 62 / 74
total count: 92
----------------------------
Now processing: 0000000000000000-180717-153000-154500-10u402000000.avi 63 / 74
total count: 92
----------------------------
Now processing: 0000000000000000-180717-154500-160000-10u402000000.avi 64 / 74
total count: 92
----------------------------
Now processing: 0000000000000000-180717-160000-161500-10u402000000.avi 65 / 74
total count: 92
----------------------------
Now processing: 0000000000000000-180717-161500-163000-10u402000000.avi 66 / 74
total count: 92
----------------------------
Now processing: 0000000000000000-180717-163000-164500-10u402000000.avi 67 / 74
[array([

error: OpenCV(3.4.4) C:\projects\opencv-python\opencv\modules\core\src\arithm.cpp:663: error: (-209:Sizes of input arguments do not match) The operation is neither 'array op array' (where arrays have the same size and the same number of channels), nor 'array op scalar', nor 'scalar op array' in function 'cv::arithm_op'
